# faces
We can extract a few more features from each frame by using the `face_recognition` library to find and analyze faces.

In [1]:
import face_recognition
import os
from vision_features_io import *

# Finding Faces
We can load an image and find the locations of each detected face. We've defined a function to simplify this called `face_locations` in `vision_features_io.py`. The number of faces can be stored as a feature in the frame-level DataFrame.

In [2]:
film = 'booksmart'
frame = 170
frame_folder = os.path.join('../frame_per_second', film)
img_path = frame_folder + '/' + film + '_frame' + str(frame) + '.jpg'

In [3]:
image = face_recognition.load_image_file(img_path)
locations = face_recognition.face_locations(image, number_of_times_to_upsample=1)

print('Found ' + str(len(locations)) + ' face(s) in frame ' + str(frame))

Found 1 face(s) in frame 170


# Face Locations
When we use `face_recognition.face_locations()` to find the locations, we're returned with a square bounding box which encompasses a character's face. We can see if either of the two rule-of-thirds points falls within this bounding box.

In [4]:
locations = face_locations('booksmart', 170)

Found 1 face(s) in frame 170


In this example, we've located a single face. A list is returned, and we can access the bounding box by accessing the first (and only) item. The bounding box has four values: top, right, bottom, left edges of the bounding box, measured in pixels, starting from the top-left corner.

In [5]:
character = locations[0] # top, right, bottom, left
character

(82, 411, 211, 282)

## Thirds Points
We can also identify where faces are present in the frame. Typically in a dialogue scene, a cinematographer will position the camera so a character lines up with one of two rule-of-thirds points. Instead of positioning a character so they're smack-dab in the center of frame, it's more dynamic if they're at two-thirds of the frame height, and one-third width away from the left- or right-edge.

We can calculate the coordinates for the left and right thirds points using a previously-defined function in the `dimensions` notebook.
- Left third point: (one-third the frame width from the left, one-third the frame height from the top)
- Right third point: (two-thirds the frame width from the left, one-third the frame height from the top)

In [6]:
frame = load_frame('booksmart', 170)
left_thirds_point = thirds_points(frame)[0]
right_thirds_point = thirds_points(frame)[1]
print(left_thirds_point)
print(right_thirds_point)

(285, 119)
(569, 119)


We'll check if this face is aligned at the left thirds point. We take the X and Y coordinates of the left thirds point and see if it lies within the bounding box of the face. 

In [7]:
if character[0] <= left_thirds_point[1] <= character[2]: # bottom of face bounding box < left thirds point X < top of face bounding box
    print('face is vertically aligned at left thirds point')

face is vertically aligned at left thirds point


In [8]:
if character[3] <= left_thirds_point[0] <= character[1]: # right edge of face bounding box < left thirds point Y < left edge of face bounding box
    print('face is horizontally aligned at left thirds point')

face is horizontally aligned at left thirds point


We can combine these two code blocks and turn them into two functions, one each for the left and right thirds points.

In [9]:
if character[0] <= left_thirds_point[1] <= character[2] and character[3] <= left_thirds_point[0] <= character[1]:
    print('face is at left thirds point')

face is at left thirds point


In [10]:
if character[0] <= right_thirds_point[1] <= character[2] and character[3] <= right_thirds_point[0] <= character[1]:
    print('face is at right thirds point')

# Primary Character
If there are multiple characters onscreen, it's helpful to know if one of them is the primary focus of the shot. For now, we can declare that a frame has a primary character is in the foreground, or closer to the camera, in a sense. If their face (and face's bounding box) is significantly larger than the other faces in frame, we can say that they're the primary character.

When we get a list of face locations, they're sorted by size. The first face in the list is always the largest.

In [11]:
locations = face_locations('booksmart', 157)

Found 2 face(s) in frame 157


When we get a list of face locations, they're sorted by size. The first face in the list is always the largest. Since the face bounding box is a square, we can compare their sizes by just looking at their width.

In [12]:
len(locations) # two faces

2

In [13]:
locations[0][1] - locations[0][3] # box right edge - box right edge

155

In [14]:
locations[1][1] - locations[1][3]

155

These characters's faces have the same width, so we won't declare a primary character.

In the next example, we have a frame with three characters. One is in the driver seat of a car, speaking, and the other two are in the backseat. We should declare the driver the primary character. 

First we'll `pop` the first face from the list, since we know it's the largest. Then we'll want to see if it's significantly larger than the other faces. We've set a threshold of 75%: if the other faces' widths aren't greater than 75% of the first face's width, we delcare the first face the primary character of the frame.

In [15]:
locations = face_locations('booksmart', 2195)

Found 3 face(s) in frame 2195


In [16]:
first_face = locations.pop(0)
first_face_size = first_face[2] - first_face[0]
primary_char_flag = 1
primary_char_threshold = .75 # 75% of the first face
for face in locations:
    if face[2] - face[0] > (first_face_size * primary_char_threshold):
        primary_char_flag = 0
        
primary_char_flag

1